# Capstone Project Battle of Neighbourhood Week 3 Part 1

In [ ]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude

In [1]:
# Lets start by importing all the modules and declaring the global variables
import pandas as pd
import numpy as np
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
import requests
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
pd.set_option('display.max_colwidth', -1)
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Folium installed and imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

## Scrape and Explore Dataset

In [2]:
tables = pd.read_html(link,header=0)[0]
tables

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


In [3]:
# Lets see the shape of dataset
tables.shape

(180, 3)

In [4]:
# Renaming the columns
tables.columns = ['PostalCode', 'Borough', 'Neighborhood']
# Extracting Table that has an assigned borrow
table_b_a = tables[tables['Borough'] != 'Not assigned']
# Extracting Table that has not assigned borrow
table_b_na = tables[tables['Borough'] == 'Not assigned']

In [5]:
# Lets check out the shape for each assigned and not assigned borrow
print(table_b_a.shape)
print(table_b_na.shape)

(103, 3)
(77, 3)


## Group by postcode

In [6]:
# Group by postcode
table_grouped = table_b_a.groupby('PostalCode')
#  These are the keys by which the table is grouped
table_grouped.groups.keys()

dict_keys(['M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M', 'M1N', 'M1P', 'M1R', 'M1S', 'M1T', 'M1V', 'M1W', 'M1X', 'M2H', 'M2J', 'M2K', 'M2L', 'M2M', 'M2N', 'M2P', 'M2R', 'M3A', 'M3B', 'M3C', 'M3H', 'M3J', 'M3K', 'M3L', 'M3M', 'M3N', 'M4A', 'M4B', 'M4C', 'M4E', 'M4G', 'M4H', 'M4J', 'M4K', 'M4L', 'M4M', 'M4N', 'M4P', 'M4R', 'M4S', 'M4T', 'M4V', 'M4W', 'M4X', 'M4Y', 'M5A', 'M5B', 'M5C', 'M5E', 'M5G', 'M5H', 'M5J', 'M5K', 'M5L', 'M5M', 'M5N', 'M5P', 'M5R', 'M5S', 'M5T', 'M5V', 'M5W', 'M5X', 'M6A', 'M6B', 'M6C', 'M6E', 'M6G', 'M6H', 'M6J', 'M6K', 'M6L', 'M6M', 'M6N', 'M6P', 'M6R', 'M6S', 'M7A', 'M7R', 'M7Y', 'M8V', 'M8W', 'M8X', 'M8Y', 'M8Z', 'M9A', 'M9B', 'M9C', 'M9L', 'M9M', 'M9N', 'M9P', 'M9R', 'M9V', 'M9W'])

In [7]:
# Lets check out one of the group
table_grouped.get_group('M5G')

,PostalCode,Borough,Neighborhood
40,M5G,Downtown Toronto,Central Bay Street


## Join the Neigborhood in a group by comma

### Creating a new empty dateframe

In [8]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood']

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
neighborhoods

,PostalCode,Borough,Neighborhood


In [10]:
def add_comma_neighbor(grp):
    '''
    This function return the borough as well as Neighbor hood for each group combined or joined with comma.
    '''
    return table_grouped.get_group(grp)['Borough'].iloc[0], ", ".join(table_grouped.get_group(grp)['Neighborhood'])

In [11]:
# Looping for each group and appending the result to our empty neighborhoods dataframe just created
for grp in table_grouped.groups.keys():
    borough, neighbor = add_comma_neighbor(grp)
    neighborhoods = neighborhoods.append({'PostalCode' : grp,
                                          'Borough': borough,
                                          'Neighborhood': neighbor
                                         }, ignore_index=True)
neighborhoods

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


In [12]:
# Lets verify our results
neighborhoods[neighborhoods['PostalCode'] == "M9V"]

,PostalCode,Borough,Neighborhood
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens


## Assigning Borough to neighborhood if a cell has a borough but a Not assigned neighborhood

In [13]:
# Lets check which are the rows that have Neighborhood as "Not assigned"
neighborhoods[neighborhoods['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


In [14]:
neighborhoods.loc[neighborhoods['Neighborhood'] == 'Not assigned', 'Neighborhood'] = neighborhoods[neighborhoods['Neighborhood'] == 'Not assigned']['Borough']

In [15]:
neighborhoods[neighborhoods['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


In [16]:
# Lets verify
neighborhoods[neighborhoods['PostalCode'] == 'M7A']

,PostalCode,Borough,Neighborhood
85,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


#### Yes we were succesfull in our operation, Lets check the shape now

In [17]:
neighborhoods.shape

(103, 3)

# PART 2

In [18]:
# Lets have a look at our database
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [19]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df.columns = ['PostalCode', 'Latitude', 'Longitude']
df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
neigh_latlong = neighborhoods.merge(df, how='inner', left_on=['PostalCode'], right_on=['PostalCode'])
neigh_latlong.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [22]:
neigh_latlong[neigh_latlong['PostalCode'] == 'M9L']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
96,M9L,North York,Humber Summit,43.756303,-79.565963


# PART 3

In [23]:
# Explore and cluster the neighborhoods in Toronto. Working with only boroughs that contain the word Toronto 
neigh_latlong = neigh_latlong[neigh_latlong['Borough'].str.contains("Toronto", case=False)]
neigh_latlong.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
42,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [24]:
# Lets get the latitude and longitude of toronto to draw a map first
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [25]:
## Lets visualise the neighborhood using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(neigh_latlong['Latitude'], neigh_latlong['Longitude'], neigh_latlong['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [26]:
CLIENT_ID = 'WPTSXRBGDLUULN1OWDRCQUAE3KIEDUNMQM5AW5Q0DH5UCCAY' # your Foursquare ID
CLIENT_SECRET = 'TR4030IUWOWEHNIZCT0EQGVC2BMLZV35KSHH404AAWSMFXAU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 1000
LIMIT = 200

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
neigh_latlong.shape

(39, 5)

In [32]:
toronto_venues = getNearbyVenues(names=neigh_latlong['Neighborhood'],
                                   latitudes=neigh_latlong['Latitude'],
                                   longitudes=neigh_latlong['Longitude']
                                  )

In [33]:
print(toronto_venues.shape)
toronto_venues.head()

(1661, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth West / Riverdale,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [34]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
Brockton / Parkdale Village / Exhibition Place,22,22,22,22,22,22
Business reply mail Processing CentrE,17,17,17,17,17,17
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,16,16,16,16,16,16
Central Bay Street,73,73,73,73,73,73
Christie,17,17,17,17,17,17
Church and Wellesley,80,80,80,80,80,80
Commerce Court / Victoria Hotel,100,100,100,100,100,100
Davisville,35,35,35,35,35,35


### Let's find out how many unique categories can be curated from all the returned venues

In [35]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


## Analyze Each Neighborhood

In [36]:
# Convert each category to one hot
toronto_one_hot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# Add neighborhood back
toronto_one_hot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
toronto_one_hot = toronto_one_hot.set_index('Neighborhood').reset_index()

toronto_one_hot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Danforth West / Riverdale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
toronto_grouped = toronto_one_hot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.017857,0.0,0.0,0.000000,0.0,0.0,0.000000
1,Brockton / Parkdale Village / Exhibition Place,0.0,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,Business reply mail Processing CentrE,0.0,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
3,CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,0.0,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
4,Central Bay Street,0.0,0.0000,0.0000,0.000,0.0000,0.000,0.013699,0.0,0.0,...,0.0,0.0,0.0,0.013699,0.0,0.0,0.013699,0.0,0.0,0.013699


In [38]:
# Lets check the new size
toronto_grouped.shape

(38, 235)

In [39]:
# Lets print each neighboorhood with the top 5 most common venue
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0  Coffee Shop         0.05
1  Cocktail Bar        0.05
2  Café                0.04
3  Italian Restaurant  0.04
4  Seafood Restaurant  0.04


----Brockton / Parkdale Village / Exhibition Place----
            venue  freq
0  Café            0.14
1  Breakfast Spot  0.09
2  Coffee Shop     0.09
3  Pet Store       0.05
4  Intersection    0.05


----Business reply mail Processing CentrE----
              venue  freq
0  Pizza Place       0.06
1  Garden Center     0.06
2  Recording Studio  0.06
3  Restaurant        0.06
4  Butcher           0.06


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport----
              venue  freq
0  Airport Service   0.19
1  Airport Lounge    0.12
2  Airport Terminal  0.12
3  Harbor / Marina   0.06
4  Coffee Shop       0.06


----Central Bay Street----
                 venue  freq
0  Coffee Shop          0.18
1  Italian Restaurant   0.07
2  Japanese

### Let's put that into a pandas dataframe

#### First, let's write a function to sort the venues in descending order.

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Coffee Shop,Café,Restaurant,Farmers Market,Bakery,Seafood Restaurant,Cheese Shop,Beer Bar,Italian Restaurant
1,Brockton / Parkdale Village / Exhibition Place,Café,Breakfast Spot,Coffee Shop,Convenience Store,Burrito Place,Restaurant,Italian Restaurant,Intersection,Stadium,Bar
2,Business reply mail Processing CentrE,Park,Auto Workshop,Comic Shop,Pizza Place,Butcher,Recording Studio,Restaurant,Burrito Place,Brewery,Light Rail Station
3,CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,Airport Service,Airport Lounge,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Coffee Shop,Rental Car Location,Harbor / Marina
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Spa,Salad Place,Bubble Tea Shop,Gym / Fitness Center,Ice Cream Shop


## Cluster Neighborhoods

In [42]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neigh_latlong

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0.0,Trail,Health Food Store,Pub,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Yoga Studio,Cupcake Shop
41,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188,0.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Restaurant,Lounge,Spa,Juice Bar
42,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572,0.0,Sandwich Place,Brewery,Burrito Place,Fast Food Restaurant,Steakhouse,Pub,Fish & Chips Shop,Sushi Restaurant,Italian Restaurant,Food & Drink Shop
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0.0,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Sandwich Place,Cheese Shop,Pet Store,Park
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2.0,Park,Bus Line,Swim School,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0.0,Park,Convenience Store,Breakfast Spot,Sandwich Place,Food & Drink Shop,Department Store,Asian Restaurant,Hotel,Gym,Concert Hall
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0.0,Clothing Store,Coffee Shop,Yoga Studio,Salon / Barbershop,Restaurant,Rental Car Location,Café,Chinese Restaurant,Mexican Restaurant,Sporting Goods Shop
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,0.0,Sandwich Place,Dessert Shop,Gym,Sushi Restaurant,Italian Restaurant,Café,Pizza Place,Coffee Shop,Gourmet Shop,Park
48,M4T,Central Toronto,Moore Park / Summerhill East,43.689574,-79.383160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park,43.686412,-79.400049,0.0,Coffee Shop,Pub,Bank,Vietnamese Restaurant,Light Rail Station,Supermarket,Liquor Store,American Restaurant,Restaurant,Sushi Restaurant


In [49]:
# Lets check this out in the map
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 1 seems to be family area or living area

In [50]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0.0,Trail,Health Food Store,Pub,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Yoga Studio,Cupcake Shop
41,East Toronto,0.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Restaurant,Lounge,Spa,Juice Bar
42,East Toronto,0.0,Sandwich Place,Brewery,Burrito Place,Fast Food Restaurant,Steakhouse,Pub,Fish & Chips Shop,Sushi Restaurant,Italian Restaurant,Food & Drink Shop
43,East Toronto,0.0,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Sandwich Place,Cheese Shop,Pet Store,Park
45,Central Toronto,0.0,Park,Convenience Store,Breakfast Spot,Sandwich Place,Food & Drink Shop,Department Store,Asian Restaurant,Hotel,Gym,Concert Hall
46,Central Toronto,0.0,Clothing Store,Coffee Shop,Yoga Studio,Salon / Barbershop,Restaurant,Rental Car Location,Café,Chinese Restaurant,Mexican Restaurant,Sporting Goods Shop
47,Central Toronto,0.0,Sandwich Place,Dessert Shop,Gym,Sushi Restaurant,Italian Restaurant,Café,Pizza Place,Coffee Shop,Gourmet Shop,Park
49,Central Toronto,0.0,Coffee Shop,Pub,Bank,Vietnamese Restaurant,Light Rail Station,Supermarket,Liquor Store,American Restaurant,Restaurant,Sushi Restaurant
51,Downtown Toronto,0.0,Coffee Shop,Park,Bakery,Chinese Restaurant,Pizza Place,Café,Pub,Italian Restaurant,Restaurant,Butcher
52,Downtown Toronto,0.0,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Yoga Studio,Café,Pizza Place,Men's Store,Mediterranean Restaurant


### Cluster 2 contains only one data where summer camp is the most common venue

In [51]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,1.0,Garden,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


#### Cluster 3 contains only one data where garden is the most common venue

In [52]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,2.0,Park,Bus Line,Swim School,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 4 area seems to be containing mainly of restaurants, coffee shop and gift shop

In [53]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,3.0,Park,Playground,Trail,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
64,Central Toronto,3.0,Park,Jewelry Store,Trail,Sushi Restaurant,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
